In [1]:
#Reading Dataset

import pandas as pd
import os

dataset_folder = os.path.join(os.getcwd(), 'dataset')

def df_from_csv(filename):
    f = os.path.join(dataset_folder, filename)
    return pd.read_csv(f, delimiter='\t|\n|,', engine='python')

In [2]:
y_train = df_from_csv('ori_trainy.csv')
y_train.head()

,alert_key,sar_flag
0,171189,0
1,171202,0
2,171599,0
3,171737,0
4,171142,0


In [3]:
x_train = df_from_csv('ori_trainx.csv')
x_train.head()

,alert_key,date
0,171189,0
1,171202,0
2,171599,0
3,171737,0
4,171142,0


In [4]:
alert_cust = df_from_csv('ori_custinfo.csv')
alert_cust.head()

,alert_key,cust_id,risk_rank,occupation_code,total_asset,AGE
0,352249,82595ac69158ae08d34156784bdec0d9e2ca5b242b6d2a...,1,19.0,1465816.0,7
1,352253,b212d14cb35676926682b2cf849e295d948888f556c07e...,1,2.0,98177.0,2
2,352254,e5b0002791c7852644a2730abeaa893cdf14a072ef7812...,1,19.0,2052922.0,7
3,352280,74214c478dc6519fbefe4bc31693865bdcd698ab974b64...,3,15.0,201906.0,5
4,352282,0340e7611f0d82c3cb87e6194fa14bb2ccf8afbf1b3418...,1,12.0,7450.0,5


In [5]:
# Merge x and y train into one dataframe, then merge it with customer info data as whole training data

train = x_train.merge(y_train, on='alert_key', how='inner')
train = train.merge(alert_cust, on='alert_key', how='inner')
train.head()

,alert_key,date,sar_flag,cust_id,risk_rank,occupation_code,total_asset,AGE
0,171189,0,0,972ee157e63316e8a50dd489bc93730a3ee8a8959d5c6b...,1,17.0,375576.0,4
1,171202,0,0,a10ab33f90926fb18d7bb5e78034d7f04a1fbed95b7951...,3,12.0,2717416.0,2
2,171599,0,0,3433ecc068ed1c9e2f5037cab5d42d7b901e9bd624c1fb...,1,12.0,326517.0,4
3,171737,0,0,a0861608615a9365d90f4ba0a813c0ea0471987f925c8b...,3,14.0,1014759.0,4
4,171142,0,0,a39fea9aec90969fe66a2b2b4d1b86368a2d38e8b8d4bf...,3,12.0,241719.0,3


In [6]:
# check null value on whole dataset & drop if necessary

train.isnull().any()

alert_key          False
date               False
sar_flag           False
cust_id            False
risk_rank          False
occupation_code     True
total_asset        False
AGE                False
dtype: bool

In [7]:
train[train.isnull().any(axis=1)]

,alert_key,date,sar_flag,cust_id,risk_rank,occupation_code,total_asset,AGE
144,172626,5,0,0468d8aa19c0c5c8f95a583d2e6992912e4a44249ceeb7...,1,NaN,0.0,3
158,172355,5,0,d17644813c30c0a23672eefcfdc8261d3ceb1821da84a7...,1,NaN,0.0,3
227,172353,5,0,8629e2c8d7cc2a96ec9a15cc9a985f8ffe664460a32133...,0,NaN,0.0,3
320,172932,6,0,3c63843b7d42445abd475aa3358e98c5817ac25a7bb580...,1,NaN,0.0,2
691,177553,11,0,53af1e57a2397b96f7ff52cd11b80a7d668348a8dd11a9...,1,NaN,0.0,3
...,...,...,...,...,...,...,...,...
20167,322382,313,0,7ba8d3174f2153063bbe616e852e55faffd2dcd92ab375...,1,NaN,0.0,5
20372,322734,314,0,e392b3a8fc15525617c552cd3385ac8d37ae62aa7e94f8...,0,NaN,0.0,2
20450,323307,315,0,02635906e3b55964d9e42f5f8aeeb62c25599f73dfd10b...,0,NaN,0.0,2
21661,334129,330,0,bc2d8f0134895f6deaee4eb189e06d8a81a5c25443559f...,1,NaN,0.0,4


In [8]:
train = train.dropna(how='any', axis=0)
train

,alert_key,date,sar_flag,cust_id,risk_rank,occupation_code,total_asset,AGE
0,171189,0,0,972ee157e63316e8a50dd489bc93730a3ee8a8959d5c6b...,1,17.0,375576.0,4
1,171202,0,0,a10ab33f90926fb18d7bb5e78034d7f04a1fbed95b7951...,3,12.0,2717416.0,2
2,171599,0,0,3433ecc068ed1c9e2f5037cab5d42d7b901e9bd624c1fb...,1,12.0,326517.0,4
3,171737,0,0,a0861608615a9365d90f4ba0a813c0ea0471987f925c8b...,3,14.0,1014759.0,4
4,171142,0,0,a39fea9aec90969fe66a2b2b4d1b86368a2d38e8b8d4bf...,3,12.0,241719.0,3
...,...,...,...,...,...,...,...,...
23901,352132,364,0,c49b33d1fde790ec03584672903f296e486615adfdc989...,1,19.0,3218731.0,3
23902,352125,364,0,0c2dc5fedc3689abf5ff4be14fe8fea8d23d22068297c2...,3,19.0,928963.0,3
23903,352080,364,0,1a93961c8fa830a1f32f5316b44f4964d65397f0311b11...,2,9.0,69080.0,6
23904,352075,364,0,9586b80d3ad1d74c0a1efe792cae2ecd073243fb01ed33...,3,3.0,262604.0,6


In [9]:
# checking the number of records based on sar_flag

train.groupby(['sar_flag'])['sar_flag'].count()

sar_flag
0    23556
1      234
Name: sar_flag, dtype: int64

In [10]:
# create method to analyze LR performance

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [11]:
# SMOTE to fix imbalance data (upscale minor data by creating synthetic record)
# Other variant SMOTEENN

# SMOTE
from imblearn.over_sampling import SMOTE

smote_x = train[['risk_rank','occupation_code','total_asset','AGE']]
smote_y = train.loc[:, train.columns == 'sar_flag']

sm = SMOTE(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(smote_x, smote_y, test_size=0.3, random_state=0)
sm_cols = X_train.columns

sm_data_X,sm_data_y = sm.fit_resample(X_train, y_train)
sm_data_X = pd.DataFrame(data=sm_data_X,columns=sm_cols )
sm_data_y = pd.DataFrame(data=sm_data_y,columns=['sar_flag'])
# we can Check the numbers of our data
print("length of oversampled data is ",len(sm_data_X))
print("Number of non-sar in oversampled data",len(sm_data_y[sm_data_y['sar_flag']==0]))
print("Number of sar",len(sm_data_y[sm_data_y['sar_flag']==1]))
print("Proportion of no subscription data in oversampled data is ", len(sm_data_y[sm_data_y['sar_flag']==0])/len(sm_data_X))
print("Proportion of subscription data in oversampled data is ", len(sm_data_y[sm_data_y['sar_flag']==1])/len(sm_data_X))

length of oversampled data is  32964
Number of non-sar in oversampled data 16482
Number of sar 16482
Proportion of no subscription data in oversampled data is  0.5
Proportion of subscription data in oversampled data is  0.5


In [12]:
# RFE feature selection > issue on selecting total asset which cause overfitting instead

from sklearn.feature_selection import RFE

LR = LogisticRegression(max_iter=2000, n_jobs=-1)
rfe = RFE(LR, step=100)
rfe = rfe.fit(sm_data_X, sm_data_y.values.ravel())
print(sm_data_X.columns)
print(rfe.support_)
print(rfe.ranking_)

Index(['risk_rank', 'occupation_code', 'total_asset', 'AGE'], dtype='object')
[False False  True  True]
[2 2 1 1]


In [13]:
# assess feature performance, P>|z| should be acceptable

import statsmodels.api as sm

updated_cols = ['risk_rank', 'AGE']
new_X = sm_data_X[updated_cols]
new_Y = sm_data_y['sar_flag']
LR2 = sm.Logit(new_Y,new_X)
result=LR2.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.671804
         Iterations 5
                          Results: Logit
Model:              Logit            Pseudo R-squared: 0.031      
Dependent Variable: sar_flag         AIC:              44294.7081 
Date:               2022-12-08 19:26 BIC:              44311.5145 
No. Observations:   32964            Log-Likelihood:   -22145.    
Df Model:           1                LL-Null:          -22849.    
Df Residuals:       32962            LLR p-value:      6.0223e-308
Converged:          1.0000           Scale:            1.0000     
No. Iterations:     5.0000                                        
-------------------------------------------------------------------
               Coef.   Std.Err.     z      P>|z|    [0.025   0.975]
-------------------------------------------------------------------
risk_rank     -0.4715    0.0144  -32.7006  0.0000  -0.4997  -0.4432
AGE            0.1208    0.0060   20.2239  0.0000

In [14]:
# Train and test

from sklearn import metrics

x_train, x_test, y_train, y_test = train_test_split(new_X, new_Y, test_size=0.25)
LR.fit(x_train, y_train)
y_train_predict = LR.predict(x_train)
y_test_predict = LR.predict(x_test)

# output = pd.DataFrame(y_test_predict, columns = ['sar_flag'])
# output.to_csv('y_test_predict.csv', index=False)

train_acc = accuracy_score(y_train, y_train_predict)
test_acc = accuracy_score(y_test, y_test_predict)
print(f'{train_acc}  {test_acc}')
cm = confusion_matrix(y_true=y_test, y_pred=y_test_predict)
print(cm)

0.6701047607491001  0.665331877199369
[[1985 2107]
 [ 651 3498]]


In [15]:
# Apply to our real dataset

x = train[['risk_rank', 'AGE']]
y = train['sar_flag']

y_predict = LR.predict(x)
test_acc = accuracy_score(y, y_predict)
print(f'{train_acc}  {test_acc}')
cm = confusion_matrix(y_true=y, y_pred=y_predict)
print(cm)

0.6701047607491001  0.5016813787305591
[[11760 11796]
 [   59   175]]
